In [1]:
import time
import numpy as np
import pandas as pd
import featuretools as ft
import catboost
from tqdm import tqdm_notebook
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.optimize
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, f1_score
from sklearn.externals import joblib
from category_encoders import CatBoostEncoder

## 資料讀取與處理

In [2]:
train_raw = pd.read_csv("train_pre_31.csv")
X_test = pd.read_csv("test_pre_31.csv")
train_raw = train_raw.drop_duplicates(subset=train_raw.columns[~train_raw.columns.isin([
        'acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'mcc',
       'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', "loctm_min", 
       'conam_mean_by_loctm_min', 
       'conam_mean_by_loctm_sec', 
       'bacno_cano_no_recent_10_conam_pos_trend',
       'bacno_cano_no_recent_7_loctm_diff', 
       'bacno_cano_loctm_uique_count', 
       'bacno_cano_14_rolling_times_prime_time_fraud_cnt',
       'bacno_cano_10_times_sleep_time_cnt',
       'bacno_cano_10_sleep_time_conam_lower_3000_cnt',
       'bacno_cano_10_prime_time_fraud_conam_lower_3000_cnt', 
       'bacno_cano_no_recent_10_ecfg_sum', 'fraud_ind', "locdt",
        'flbmk_value_counts', 'hcefg_value_counts', 'flg_3dsmk_value_counts'
         'ecfg_value_counts', 'ovrlt_value_counts', 'insfg_value_counts'
        "conam_mean_by_train_test_acqic", "loctm_value_counts",
        "loctm_hour_value_counts", 'loctm_sec'])], keep='last')
test_raw = pd.read_csv("test.csv")

X = train_raw.drop('fraud_ind', axis=1)
y = train_raw['fraud_ind']

print(train_raw.shape)
print(X_test.shape)
print(test_raw.shape)

(1461618, 115)
(421665, 114)
(421665, 22)


In [3]:
all_features = pd.concat([X, X_test], axis=0)
all_features.tail()

del all_features["locdt"]
del all_features['flg_3dsmk_value_counts']
del all_features['flbmk_value_counts']
del all_features['hcefg_value_counts']
del all_features['ecfg_value_counts']
del all_features['ovrlt_value_counts']
del all_features['insfg_value_counts']
del all_features["conam_mean_by_train_test_acqic"]

del all_features["loctm_value_counts"]
del all_features["loctm_hour_value_counts"]
del all_features['loctm_sec']

X = all_features[:-len(X_test)]
X_test = all_features[-len(X_test):]
print(X.shape)
print(X_test.shape)

(1461618, 103)
(421665, 103)


## 定義資料type

In [4]:
cat_colnames = ["bacno", "cano", "mchno", "acqic", "mcc", "ecfg", "insfg", "contp", 'etymd',
                "stocn", "scity", "stscd", "ovrlt", "flbmk", "hcefg", "csmcu", "flg_3dsmk"] # + high_cacol

In [5]:
# Your data contains string(?) values. You should set categorical columns in pandas to treat them properly. 
# After that there is no need to pass them again in categorical_feature param. 
model_type = 'lgb'
for col in cat_colnames:
    # for catboost
    if model_type == 'cat':
        try:
            X[col] = X[col].astype('str')
            X_test[col] = X_test[col].astype('str')
            print(col)
            print(X[col].dtype)
        except Exception:
            print("No ", col, " exists")
    else:
        try:
            X[col] = X[col].astype('category')
            X_test[col] = X_test[col].astype('category')
            print(col)
            print(X[col].dtype)
        except Exception:
            print("No ", col, " exists")
# X['city_town'] = X['city_town'].astype('str')

bacno
category
cano
category
mchno
category
acqic
category
mcc
category
ecfg
category
insfg
category
contp
category
etymd
category
stocn
category
scity
category
stscd
category
ovrlt
category
flbmk
category
hcefg
category
csmcu
category
flg_3dsmk
category


## 資料Split

In [6]:
perm = np.random.permutation(len(X)) 
X = X.iloc[perm].reset_index(drop=True) 
y = y.iloc[perm].reset_index(drop=True)

sss = StratifiedKFold(n_splits=4, random_state=111, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

Train: [ 357931  357948  358029 ... 1461615 1461616 1461617] Test: [     0      1      2 ... 365485 365486 365487]
Train: [      0       1       2 ... 1461615 1461616 1461617] Test: [357931 357948 358029 ... 730833 730834 730835]
Train: [      0       1       2 ... 1461615 1461616 1461617] Test: [ 728957  729015  729104 ... 1096284 1096285 1096286]
Train: [      0       1       2 ... 1096284 1096285 1096286] Test: [1089685 1089726 1089803 ... 1461615 1461616 1461617]
(1096214, 103)
(1096214,)
(365404, 103)
(365404,)


## 不同模型適配

## 模型調參

In [18]:
from hyperopt import STATUS_OK

# Domain Space
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

# Optimization Algorithm
from hyperopt import tpe, fmin, Trials

from timeit import default_timer as timer

#### Create Domain Space (parameter space)

In [32]:
# 需對每個要調的參數創造空間

"""
Demo
"""
# learning rate: 
# learning_rate = {'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2))}
# learning_rate_dist = []
# Draw 10000 samples from the learning rate domain
# for _ in range(10000):
#     learning_rate_dist.append(sample(learning_rate)['learning_rate'])  # 檢視抽樣空間結果

# num_leaves
# num_leaves = {'num_leaves': hp.quniform('num_leaves', 30, 150, 1)}
# num_leaves_dist = []

# boosting type domain
# boosting_type = {'boosting_type':
#                      hp.choice('boosting_type',
#                                [{'boosting_type': 'gbdt', 'subsample': hp.uniform('subsample', 0.5, 1)},
#                                 {'boosting_type': 'dart', 'subsample': hp.uniform('subsample', 0.5, 1)},
#                                 {'boosting_type': 'goss', 'subsample': 1.0}]
#                                )
#                  }
# Draw a sample
# params_set = sample(boosting_type)

space = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    # 'n_estimators': hp.quniform('num_leaves', 100, 5000, 1),
#     'class_weight': hp.choice('class_weight', [None, 'balanced']),
#     'boosting_type': hp.choice('boosting_type',
#                                [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)},
#                                 {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
#                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 20, 80, 1),
    # 'learning_rate': hp.loguniform('learning_rate', np.log(0.0005), np.log(0.2)),
    'learning_rate': 0.1,
    # 'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    # 'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    # 'reg_alpha': hp.uniform('reg_alpha', 0.0, 3.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 3.0),
    'feature_fraction': hp.uniform('feature_fraction', 0.6, 1.0),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.6, 1.0),
    'min_child_weight': hp.uniform('min_child_weight', 0.0, 30),
    # 'max_drop': hp.quniform('max_drop', 30, 60, 5),
    # 'skip_drop': hp.uniform('skip_drop', 0.5, 0.8),
    # 'drop_rate': hp.uniform('drop_rate', 0.1, 0.5),
    'scale_pos_weight': hp.quniform('scale_pos_weight', 1, 10, 0.3)
}

In [34]:
# Sample from the full space
example = sample(space)

# Dictionary get method with default
# subsample = example['boosting_type'].get('subsample', 1.0)

# Assign top-level keys
# example['boosting_type'] = example['boosting_type']['boosting_type']
# example['subsample'] = subsample

example

{'bagging_fraction': 0.9781067413963053,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.6289145448268182,
 'learning_rate': 0.1,
 'min_child_weight': 20.233194332365073,
 'num_leaves': 25.0,
 'objective': 'binary',
 'reg_lambda': 0.6369540662445169,
 'scale_pos_weight': 9.6}

#### Objective function

In [35]:
# N_FOLDS = 5

# Create the dataset
# train_set = lgb.Dataset(X_train_hyper, y_train_hyper)

def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    
    # 放那些只吃整數的參數
    for parameter_name in ['num_leaves']: # , 'max_drop'
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
#     cv_results = lgb.cv(params, 
#                         train_set, 
#                         nfold=n_folds, 
#                         num_boost_round=100, # 先放1000去看~
#                         # early_stopping_rounds=100,  # not work for dart
#                         metrics='rmse', 
#                         # seed=50,
#                         stratified=False)
    
    model_gbdt = LGBMClassifier(**params)
    
    
#     model_dart = LGBMClassifier(objective='binary', # regression, L2 loss, aliases: regression_l2, l2, mean_squared_error, mse, l2_root, root_mean_squared_error, rmse
#                              boosting_type='dart',
#                              num_leaves=params['num_leaves'],  # < 2^max_depth
#                              reg_lambda=params['reg_lambda'],
#                              # reg_alpha=params['reg_alpha'],
#                              learning_rate=0.1, # 先設一樣的learning rate去調
#                              n_estimators=1000,
#                              bagging_fraction=params['bagging_fraction'],
#                              feature_fraction=params['feature_fraction'],
#                              drop_rate=params['drop_rate'],  # default=0.1
#                              max_drop=params['max_drop'], # 10  # default=50, 每iteration最多丟掉幾顆樹
#                              skip_drop=params['skip_drop'], # 0.6 # default=0.5 每iteration決定dropout的機率
#                              # xgboost_dart_mode=True,  # default=False, 是否要用xgb的dart模式
#                              # uniform_drop=True,  # set this to true, if you want to use uniform drop
#                              # drop_seed=11,  # random seed to choose dropping models
#                              # early_stopping_rounds=100
#                              # feature_fraction_seed=7
#                          )

    print_callback = lgb.print_evaluation(period=1000)

    fit_result = model_gbdt.fit(X_train, y_train, 
                  eval_set=[(X_val, y_val)],
                  eval_metric='auc',
                  # categorical_feature=categorical_feature_idx,
                  # early_stopping_rounds=100, # 在dart不能用，因為每一輪整個模型參數都會被更新
                  verbose=0,
                  callbacks=[print_callback]
                 )
    
    
    # print(cv_results)
    # stratify works only with classification problems. So to work with regression, you need to make it False.
    
    run_time = timer() - start
    
    # Extract the best score
    loss = fit_result.best_score_['valid_0']['binary_logloss']
    
    # Boosting rounds that returned the highest cv score
    # n_estimators = int(np.argmax(cv_results['rmse-mean']) + 1)
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, run_time])
    of_connection.close()
    
    # Loss must be minimized
    # loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'train_time': run_time, 'status': STATUS_OK}

#### Optimization Algorithm

In [36]:
# Algorithm
tpe_algorithm = tpe.suggest

4.0.4  Result History

Keeping track of the results is not strictly necessary as Hyperopt will do this internally for the algorithm. However, if we want to find out what is going on behind the scenes, we can use a Trials object which will store basic training information and also the dictionary returned from the objective function (which includes the loss and params). Making a trials object is one line:

In [37]:
# Keep track of results
bayes_trials = Trials()

In [39]:
import csv

# File to save first results
out_file = 'lgb_gbdt_trials_pre_31_lr_0_1_fix.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'train_time'])
of_connection.close()

#### Optimization

In [40]:
from hyperopt import fmin

MAX_EVALS = 200

# Optimize
best = fmin(fn=objective, space=space, algo=tpe.suggest,
            max_evals=MAX_EVALS, trials=bayes_trials)

100%|██████████| 200/200 [3:31:01<00:00, 43.28s/it, best loss: 0.015268564481717449]   


In [41]:
# lr固定(0.1)、樹數量固定(1000)的情況下的結果
bayes_trials_results = sorted(bayes_trials.results, key=lambda x: x['loss'])
print(bayes_trials_results[:2])

[{'loss': 0.015268564481717449, 'params': {'bagging_fraction': 0.7795345629576638, 'boosting_type': 'gbdt', 'feature_fraction': 0.793185302484529, 'learning_rate': 0.1, 'min_child_weight': 8.788416143799257, 'num_leaves': 79, 'objective': 'binary', 'reg_lambda': 0.9988765862420731, 'scale_pos_weight': 0.8999999999999999}, 'train_time': 40.911826491355896, 'status': 'ok'}, {'loss': 0.015305418125836759, 'params': {'bagging_fraction': 0.62306323285472, 'boosting_type': 'gbdt', 'feature_fraction': 0.8607417975146786, 'learning_rate': 0.1, 'min_child_weight': 11.764607875190393, 'num_leaves': 77, 'objective': 'binary', 'reg_lambda': 0.46657449147625035, 'scale_pos_weight': 0.8999999999999999}, 'train_time': 39.90269403811544, 'status': 'ok'}]


#### retrain the model

In [9]:
# lightGBM (dart)
{'bagging_fraction': 0.9292807078881113, 'boosting_type': 'gbdt', 'feature_fraction': 0.8601841783747871, 
 'learning_rate': 0.06303137412922537, 'num_leaves': 61, 'objective': 'binary', 'reg_alpha': 2.1700796673636176,
 'reg_lambda': 1.256856046624738, 'scale_pos_weight': 0.8999999999999999}

model_gbdt_tuned = LGBMClassifier(objective='binary', # regression, L2 loss, aliases: regression_l2, l2, mean_squared_error, mse, l2_root, root_mean_squared_error, rmse
                             boosting_type='gbdt',
                             num_leaves=50,  # < 2^max_depth
                             reg_lambda=1.25,
                             # reg_alpha=2.17,
                             learning_rate=0.06, # 0.1
                             n_estimators=3500,
                             # max_bin=200, # default=255
                             bagging_fraction=0.75,
                             # bagging_freq=5, 
                             # bagging_seed=7,
                             feature_fraction=0.75,
                             # drop_rate=0.1184,  # default=0.1
                             # max_drop=50, # 10  # default=50, 每iteration最多丟掉幾顆樹
                             # skip_drop=0.7283, # 0.6 # default=0.5 每iteration決定dropout的機率
                             scale_pos_weight=1.8,
                             min_child_weight= 8.788416143799257,
                             # xgboost_dart_mode=True,  # default=False, 是否要用xgb的dart模式
                             # uniform_drop=True,  # set this to true, if you want to use uniform drop
                             # drop_seed=11,  # random seed to choose dropping models
                             # early_stopping_rounds=100
                             # feature_fraction_seed=7
                         )

print_callback = lgb.print_evaluation(period=250)

result = model_gbdt_tuned.fit(X, y, 
              # eval_set=[(X_val, y_val)],
              # eval_metric='auc',
              # categorical_feature=categorical_feature_idx,
              # early_stopping_rounds=100, # 在dart不能用，因為每一輪整個模型參數都會被更新
              verbose=0,
              callbacks=[print_callback]
             )


In [ ]:
# save model
joblib.dump(model_gbdt_tuned, 'model_gbdt_tuned_pre_31_final.pkl')
# load model
# gbm_pickle = joblib.load('lgb.pkl')

In [10]:
test_pred = model_gbdt_tuned.predict(X_test)
test_pred.sum()

5931

In [11]:
submission = pd.concat([test_raw["txkey"], pd.Series(test_pred, name='fraud_ind')], axis=1)
submission.to_csv("submission_78_model_gbdt_tuned_full_min_child.csv", index=None)